In [2]:
import pyreadr
import pandas as pd
from sklearn.experimental import enable_iterative_imputer

In [7]:
# Replace with your file path
test_dat = pyreadr.read_r('Intermediate_data/ver1/test_dat_before_imputing.RData')

In [8]:
# The file may contain one or more R objects (e.g., data frames, lists)
# result is a dictionary where keys are the names of the objects in the RData file
print(test_dat.keys())

odict_keys(['test_dat'])


In [9]:
# Extract one object (for example, the first data frame)
test_dat = next(iter(test_dat.values()))

In [10]:
# Verify it's a pandas DataFrame
print(type(test_dat))
print(test_dat.head())

<class 'pandas.core.frame.DataFrame'>
   NCOMP  ETAMi  SESSO  STCIVMi AMATR  ISTRMi  CONDMi LAVPAS POSIZMi  CITTMi  \
0      2     12      2        2  1991       9       3      1     NaN       1   
1      4     11      2        2  2001       7       1    NaN       2       1   
2      4      7      1        1   NaN       7       2      1     NaN       1   
3      2     15      2        2  1970       7       3      1     NaN       1   
4      4     10      2        2  2013       7       3      2       2       1   

   ... PPAPO PSIND PGRVO PAECO PCULT  PASPRO  INCSPI  BMI  RISEC   ID  
0  ...   NaN   NaN   NaN   NaN   NaN     NaN       1    2      2  1_2  
1  ...     1     3     5     1     3       5       1    2      2  2_2  
2  ...     1     3     5     1     3       5       2    2      2  2_4  
3  ...     1     3     5     1     3       5       1    3      2  3_2  
4  ...     1     3     5     1     3       5       1    2      3  4_2  

[5 rows x 103 columns]


In [12]:
train_dat = pyreadr.read_r('Intermediate_data/ver1/train_dat_before_imputing.RData')
train_dat = next(iter(train_dat.values()))
print(type(train_dat))
print(train_dat.head())

<class 'pandas.core.frame.DataFrame'>
   NCOMP  ETAMi  SESSO  STCIVMi AMATR  ISTRMi  CONDMi LAVPAS POSIZMi  CITTMi  \
0      2     12      1        2  1991       7       2      2       2       1   
1      4     11      1        2  2001       7       1    NaN       2       1   
2      4      8      1        1   NaN       7       2      1     NaN       1   
3      2     15      1        2  1970       7       3      2       2       1   
4      4     10      1        2  2013       9       1    NaN       3       1   

   ... PPAPO PSIND PGRVO PAECO PCULT  PASPRO  INCSPI  BMI  RISEC   ID  
0  ...     1     3     5     1     3       5       1    3      2  1_1  
1  ...     1     3     5     1     3       5       1    2      2  2_1  
2  ...     1     3     5     1     3       5       2    2      2  2_3  
3  ...     1     3     5     1     3       5       1    2      2  3_1  
4  ...     1     3     5     1     3       5       1    3      3  4_1  

[5 rows x 103 columns]


In [15]:
from sklearn.impute import IterativeImputer

In [16]:
# Step 3: Define the columns you want to impute
cols_to_impute = [
    "INFOR", "LING", "ARTCUL", "AMICI", "PARENT", "AMICI2", "VICINI", "PANPAS", "SALUMI", "POLLO",
    "COV", "CBOV", "CMAIAL", "LATTE", "FORM", "UOVA", "PESCE", "VERD", "POMOD", "FRUTTA", "LEGUMI", "DOLCI",
    "BGAS", "BIRRA", "VINO", "BFPAS", "ALCOL", "AMAR", "LIQUOR", "FUMO", "CRONI", "LIMITA", "FREQSERIE", "TEATRO",
    "CINE", "MUSEO", "MUSIC", "ACMUS", "SPSPO", "DISCO", "MONUM", "FREQIN12", "LQUOT", "QUONLINE", "CHIES", "SITEC",
    "SALUT", "RELFAM", "RELAM", "TEMLIB", "AMBIENTE", "PPAPO", "PSIND", "PGRVO", "PAECO", "PCULT", "PASPRO", "INCSPI"
]

# Step 4: Initialize the IterativeImputer
imputer = IterativeImputer(
    n_nearest_features=10,
    initial_strategy='median',
    max_iter=300,
    random_state=42
)

In [17]:
# Step 5: Define a function to impute only selected columns
def impute_selected_columns(df, cols, imputer):
    # Select only the subset of columns to impute
    subset = df[cols]
    
    # Fit and transform only those columns
    imputed_subset = pd.DataFrame(imputer.fit_transform(subset), columns=subset.columns, index=subset.index)
    
    # Replace the original columns with the imputed ones
    df[cols] = imputed_subset
    return df

# Step 6: Apply imputation separately to each datasettrain_dat
train_dat_imputed = impute_selected_columns(train_dat.copy(), cols_to_impute, imputer)
test_dat_imputed = impute_selected_columns(test_dat.copy(), cols_to_impute, imputer)

print(type(train_dat_imputed))
print(train_dat_imputed.head())

print(type(test_dat_imputed))
print(test_dat_imputed.head())

/opt/miniconda3/envs/impute_rdata_env/lib/python3.11/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
   NCOMP  ETAMi  SESSO  STCIVMi AMATR  ISTRMi  CONDMi LAVPAS POSIZMi  CITTMi  \
0      2     12      1        2  1991       7       2      2       2       1   
1      4     11      1        2  2001       7       1    NaN       2       1   
2      4      8      1        1   NaN       7       2      1     NaN       1   
3      2     15      1        2  1970       7       3      2       2       1   
4      4     10      1        2  2013       9       1    NaN       3       1   

   ...  PPAPO  PSIND  PGRVO PAECO PCULT  PASPRO  INCSPI  BMI  RISEC   ID  
0  ...    1.0    3.0    5.0   1.0   3.0     5.0     1.0    3      2  1_1  
1  ...    1.0    3.0    5.0   1.0   3.0     5.0     1.0    2      2  2_1  
2  ...    1.0    3.0    5.0   1.0   3.0     5.0     2.0    2      2  2_3  
3  ...    1.0    3.0    5.0   1.0   3.0     5.0     1.0    2      2  3_1  
4  ...    1.0    3.0    5.0   1.0   3.0     5.0     1.0    3      3  4_1  

[5 rows x 103 columns]
<class 

/opt/miniconda3/envs/impute_rdata_env/lib/python3.11/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [18]:
# Step 7: Save the imputed data to new CSV files
train_dat_imputed.to_csv("Intermediate_data/ver1/train_dat_imputed.csv", index=False)
test_dat_imputed.to_csv("Intermediate_data/ver1/test_dat_imputed.csv", index=False)

print("Imputation complete! Only the selected columns were imputed.")

Imputation complete! Only the selected columns were imputed.
